<a href="https://colab.research.google.com/github/geoffwoollard/ece1512_project/blob/master/nb/cryo_Residual_Network_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO
- GPU to TPU? https://www.kdnuggets.com/2019/03/train-keras-model-20x-faster-tpu-free.html
- tip from [blog post](https://dev.to/kriyeng/8-tips-for-google-colab-notebooks-to-take-advantage-of-their-free-of-charge-12gb-ram-gpu-be4): to speed up move data to root directory
- weights from another task (transfer learning)
- [add gussian noise](https://towardsdatascience.com/noise-its-not-always-annoying-1bd5f0f240f)
- automate copy J??? data to root dir /
- window particles
- augment on predict. verify sim data OK
- train on sub part data (micells vs no micelle)

In [1]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=d24545bc7ab9c138051ec1e8e74cb712b246c8b34c5f36c3c764fc4fbdcdd380
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 159.4 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


In [29]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
!cp -r /content/drive/My\ Drive/mbp/mohammad/ece1512_project/data/ouhn_Data1_P1/J1028 /
#!cp -r /content/drive/My\ Drive/mbp/mohammad/ece1512_project/data/ouhn_Data1_P1/J1012 /

In [10]:
%cd /content/drive/My\ Drive/mbp/mohammad/ece1512_project
import mrc, fit_generator_helper, customizable_deep_network


/content/drive/My Drive/mbp/mohammad/ece1512_project


In [4]:
%cd /content/drive/My\ Drive/mbp/Deep-Learning/resnet_keras/
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
import cv2
from resnets_utils import *

from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

/content/drive/My Drive/mbp/Deep-Learning/resnet_keras


In [0]:
# !cp -r /content/drive/My\ Drive/mbp/mohammad/ece1512_project/data/ouhn_Data1_P1/J977 /
!cp -r /content/drive/My\ Drive/mbp/mohammad/ece1512_project/data/ouhn_Data1_P1/J930 /
# !cp -r /content/drive/My\ Drive/mbp/mohammad/ece1512_project/data/ouhn_Data1_P1/J1028 /


In [34]:
!ls /

bin					   J1028	 sbin
boot					   keras_models  srv
content					   lib		 swift
datalab					   lib32	 sys
dev					   lib64	 tensorflow-1.15.2
dlib-19.18.0-cp27-cp27mu-linux_x86_64.whl  media	 tmp
dlib-19.18.0-cp36-cp36m-linux_x86_64.whl   mnt		 tools
etc					   opt		 usr
home					   proc		 var
J1011					   root
J1012					   run


### Load the data set

In [0]:
labels = []
files = []
#label_d = {'J977':0,'J978':1,'J979':2} # protomer, side, top
label_d = {'J1028':0,'J930':1} # +/- micelle. different underlying picks
for jdir in label_d.keys():
  for fname in glob('/%s/downsample/*mrc' % jdir): #glob('/content/drive/My Drive/mbp/mohammad/ece1512_project/data/ouhn_Data1_P1/%s/downsample/*mrc' % jdir): 
    files.append(fname)
    labels.append(label_d[jdir])


In [39]:
df = fit_generator_helper.parse_files(files,labels)

header = mrc.read_header(df['fname'].iloc[0])
nx,ny = header['nx'], header['ny']

n = int(df['class'].value_counts().min())
df0 = df.loc[(df['class'] == 0)].sample(n)
df1 = df.loc[(df['class'] == 1)].sample(n)
#df2 = df.loc[(df['class'] == 2)].sample(n)

df = pd.concat([df0,df1])
#df = pd.concat([df0,df1,df2])


df = df.sample(df.shape[0])

df['class'].value_counts() 

1    25158
0    25158
Name: class, dtype: int64

In [40]:
# test/train split
val_n = int(0.02*df.shape[0]) 
df_val = df.iloc[-val_n:]
df = df.iloc[:-val_n]
df.shape, df_val.shape

((49310, 3), (1006, 3))

In [41]:
%%time
# 12 min 3.4k
# 13 min 3.4k
# 33s 100 
# 2.5s/3.25min 706
X_val,Y_val = fit_generator_helper.XY_from_df_batch(df_val,nx=nx,ny=ny,num_classes=df['class'].unique().size)
print(X_val.shape)

(1006, 64, 64, 1)
CPU times: user 83.3 ms, sys: 29.5 ms, total: 113 ms
Wall time: 114 ms


In [49]:
img_height,img_width = nx,ny 
num_classes = df['class'].unique().size
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,1))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [0]:
from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [52]:
batch_size = 20 # do not put this too large. 100 too big. 50 getting big, perhaps can go bigger. 20 ok
n=df.shape[0]
steps_per_epoch = np.floor(df.sample(n).shape[0] / batch_size) # 110 for 11k particles
steps_per_epoch

2465.0

In [53]:
from sklearn.metrics import confusion_matrix
from datetime import datetime
dateTimeObj = datetime.now()
!mkdir /keras_models

mkdir: cannot create directory ‘/keras_models’: File exists


In [74]:
model_fname = '/keras_models/micelle_aug_'
for epoch in range(20,40):
    print('actual epoch %i' % epoch)
    model.fit_generator(fit_generator_helper.image_loader(df.sample(n),
                                                          do_augment=True,
                                                          batch_size=batch_size,
                                                          nx=nx,ny=ny,
                                                          num_classes=df['class'].unique().size),
                        steps_per_epoch=steps_per_epoch, # steps_per_epoch is number of batches per epoch
                        epochs=1,
                        )
    
    XY_aug_list = [fit_generator_helper.batch_image_augment(X_val,Y_val,batch_size=X_val.shape[0],rotation_range=180) for x in range(3)]
    for X, Y in [(X_val, Y_val)] + XY_aug_list:
      scores = model.evaluate(X, Y)
      print ('epoch %s' % epoch, "Loss = " + str(scores[0]), "Test Accuracy = " + str(scores[1]))
      model.save(model_fname+dateTimeObj.strftime("%Y%m%d-%H%M%S")+'_ep%i.h5'%epoch)
      Y_pred = model.predict(X)
      print(confusion_matrix(Y.argmax(axis=1), Y_pred.argmax(axis=1)))

    

actual epoch 20
Epoch 1/1
1006/1006 [==============================] - 1s 725us/step
epoch 20 Loss = 1.013901698660424 Test Accuracy = 0.609343945980072
[[132 378]
 [ 15 481]]
1006/1006 [==============================] - 1s 696us/step
epoch 20 Loss = 0.3884618652625302 Test Accuracy = 0.8339959979057312
[[383 127]
 [ 40 456]]
1006/1006 [==============================] - 1s 715us/step
epoch 20 Loss = 0.390476577206113 Test Accuracy = 0.8240556716918945
[[369 141]
 [ 36 460]]
1006/1006 [==============================] - 1s 701us/step
epoch 20 Loss = 0.3989915361935291 Test Accuracy = 0.8250496983528137
[[369 141]
 [ 35 461]]
actual epoch 21
Epoch 1/1
1006/1006 [==============================] - 1s 719us/step
epoch 21 Loss = 2.271218903259065 Test Accuracy = 0.6023856997489929
[[503   7]
 [393 103]]
1006/1006 [==============================] - 1s 694us/step
epoch 21 Loss = 0.3541952266133803 Test Accuracy = 0.8389661908149719
[[400 110]
 [ 52 444]]
1006/1006 [=============================

In [54]:
model_fname = '/keras_models/micelle_aug_'
for epoch in range(20,40):
    print('actual epoch %i' % epoch)
    model.fit_generator(fit_generator_helper.image_loader(df.sample(n),
                                                          do_augment=True,
                                                          batch_size=batch_size,
                                                          nx=nx,ny=ny,
                                                          num_classes=df['class'].unique().size),
                        steps_per_epoch=steps_per_epoch, # steps_per_epoch is number of batches per epoch
                        epochs=1,
                        )
    
    XY_aug_list = [fit_generator_helper.batch_image_augment(X_val,Y_val,batch_size=X_val.shape[0],rotation_range=180) for x in range(3)]
    for X, Y in [(X_val, Y_val)] + XY_aug_list:
      scores = model.evaluate(X, Y)
      print ('epoch %s' % epoch, "Loss = " + str(scores[0]), "Test Accuracy = " + str(scores[1]))
      model.save(model_fname+dateTimeObj.strftime("%Y%m%d-%H%M%S")+'_ep%i.h5'%epoch)
      Y_pred = model.predict(X)
      print(confusion_matrix(Y.argmax(axis=1), Y_pred.argmax(axis=1)))

    

actual epoch 0
Epoch 1/1
1006/1006 [==============================] - 2s 2ms/step
epoch 0 Loss = 3.518287252953228 Test Accuracy = 0.5089462995529175
[[510   0]
 [494   2]]
actual epoch 1
Epoch 1/1
1006/1006 [==============================] - 1s 722us/step
epoch 1 Loss = 6.927720621613338 Test Accuracy = 0.49105367064476013
[[  0 510]
 [  2 494]]
actual epoch 2
Epoch 1/1
1006/1006 [==============================] - 1s 715us/step
epoch 2 Loss = 17.847723712504028 Test Accuracy = 0.4930417537689209
[[  0 510]
 [  0 496]]
actual epoch 3
Epoch 1/1
1006/1006 [==============================] - 1s 718us/step
epoch 3 Loss = 8.974554674288862 Test Accuracy = 0.5069582462310791
[[510   0]
 [496   0]]
actual epoch 4
Epoch 1/1
1006/1006 [==============================] - 1s 717us/step
epoch 4 Loss = 0.9626611518575467 Test Accuracy = 0.4930417537689209
[[  0 510]
 [  0 496]]
actual epoch 5
Epoch 1/1
1006/1006 [==============================] - 1s 729us/step
epoch 5 Loss = 12.681935247796668 Test A


|jobs | pix | train acc (%) | val acc (%) | epochs|notes|
|-|-|-|-|-|-|
|J152 vs J153, 1.5k:1.5k | 64 | 99 | 57-61 | 25-35||
|J158 vs J159 good particles from excluded 2d (sort of particles) 17k:17k |64| 97-99|59-61 | 20-29||
|J158 vs J162 junk (pretty bad) from good particles 17k:17k |64|99|99|3-6|can easily tell apart junk from good particles. train/test acc 95%/98% (sic) after one epoch. train time / epoch ranges 600-170s|
|J167 v J168 side v top 3.5k:3.5k | 64 | 98 | 50-55 | 10-15 |  over trains and flatlines at 50% val after 24 epochs. not enouch data?|
|J169 v J170 side v top with lp filter 12 A 3.5k:3.5k | 64 | 97-98| 50-55  |  15-30 |  lp filter does not help|
|ouhn Data1 P1 J996 v J997, 3.5k:3.5k IP3R top v side| 128 | 96-97 | 55-60|8-10|can not distinguish side classes that are easy in 2D|
|ouhn Data1 P1 J996 v J997, 3.5k:3.5k IP3R top v side| 128 | 70 | 50|20|+ augmentation (random angle). training slower, but rising stedily. prediction loss jumping around but accuracy constant at 50% |
|("")| 128 | 80-86 | 65-85 (outlier min 54 on epoch 31)|30-40|+ augmentation (random angle). need many more epochs to train, about epoch 25-30 validation accuracy starts to rise |
|("")| 128 | 87-90 | 52-85 |45-55|+ augmentation (random angle). validation accuracy getting consistenty better, but dips |
| ouhn Data1 P1 J930 J931 36k:36k topaz picks: high res 2d vs bad particles (second round in 2d class)|64|78-94|69-66|5-10. (ok speed at 5min/ep, 85ms/20 sample batch|no augmentation. dropout rate 50%. validation accuracy peaks around 3-4 epochs, in high 60s then decreases; overfitting|
| (")|64|63-69; 53 outlier|76-77|25-30 (ok speed at 5min/ep, 85ms/20 sample batch|+ augmentation. dropout rate 50%. validation accuracy rises from 50 around ep 14 and bounces around but increased on average. no overfitting bc training acc slowly increasing|
| ouhn Data1 P1 J940 J941 3.9k:3.9k IP3R like class from het ref -> 2d class -> really good from bad (low particles per class) |64|60|49-54|90-100 |+ augmentation. dropout rate 50%. validation accuracy mainly around 50. cant train. takes many epochs training acc to rise above 50. loss plateaus: 0.71 ep 30 0.68 ep 50 0.676 ep 75 0.665 ep 100. need more data? |
| (") |64|98-99|54-56|6-10 |no augmentation. dropout rate 50%. overtrains.validation acc best in first 5 epochs, then lowers |
|J966 J967 45k abinit -> 2 polled junk classes vs good, 9.4k:9.4k| 64|60 by ep 10; max 65 at ep 50 | 49-62 ep 40-50|ended at 50 ep|+aug. dr 50. too difficult to train? loss plateauing at |
|J977 J978 J989; 4.7k each; protomer, side tetramer, top tetramer|64|start random (0.34 acc bc 3 classes, 60 by ep 4, 90 by ep 10)|validatoin bouncing around 0.37-0.42. perhaps peaks at epoch 10| max 14 epochs|no aug. dr 50. overfitting|
|(") (three_classes)|64|50 by ep 6; 78 by ep 50|50 by ep 11; 63-67 by ep 50|50|+ aug. dr50. loss still decreasing at ep 50, train more|
|J1002 J1003. 5k each. snr 0.05. `sim_data_3mer_4mer`|64|95+ first epoch|50|10|yes aug. not overtraining since with no aug validation acc high. fails bc augmenting artefact. should also mimic augment on prediction to keep consistent|
|J1002 J1003. 5k each. snr 0.05. `sim_data_3mer_4mer_aug`|64|98.9 first epoch|99.8 first epoch|99.9 by 3 ep|no aug. not overtraining since with no aug validation acc high. fails bc augmenting artefact. should also mimic augment on prediction to keep consistent|
|J1011 J1012. 5k each. snr 0.01. sim_data_3mer_4mer_snr0.01_noaug_|64|99.2 1 ep; 99.99 by ep 2|100 1 ep|perfect by ep 1-3|no aug|
|J1028,J930. 25k each. +/- micelle. different underlying picks. `micelle_noaug_`| 64 | 90 by ep 7; 97 by ep 13 | 72 by ep 3, but stays; max 74 by ep 13;  ||no aug|


In [30]:
!ls /keras_models/

sim_data_3mer_4mer_noaug_20200420-031213_ep0.h5
sim_data_3mer_4mer_noaug_20200420-031213_ep1.h5
sim_data_3mer_4mer_noaug_20200420-031213_ep2.h5
sim_data_3mer_4mer_noaug_20200420-031213_ep3.h5
sim_data_3mer_4mer_noaug_20200420-031213_ep4.h5
sim_data_3mer_4mer_noaug_20200420-031213_ep5.h5
three_classes_20200420-030444_ep0.h5
three_classes_20200420-030444_ep1.h5
three_classes_20200420-030444_ep2.h5
three_classes_20200420-030444_ep3.h5
three_classes_20200420-030444_ep4.h5
three_classes_20200420-030444_ep5.h5


In [1]:
model.summary()

NameError: ignored